In [1]:
#!pip install "sap-llm-commons[all]"

In [2]:
#!pip install ai-api-client-sdk

In [3]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

C:\Users\I541988\AppData\Local\Temp\ipykernel_22516\3646097473.py:5: UserWarning: Starting from verison 1.0.0 the default proxy_version was set to 'gen-ai-hub'. Use gen_ai_hub.proxy.core.proxy_clients.set_proxy_version('btp') to set the proxy_version to 'btp'.
  import llm_commons.proxy.base


In [4]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [5]:
resource_group = widgets.Text(
    value='default', # resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)

resource_group

Text(value='default', placeholder='Resource group of deployments')

In [6]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [7]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'
os.environ["TAVILY_API_KEY"] = sk['tavily']

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [8]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()

In [9]:
PDF_NAME="ai.pdf"
from langchain.document_loaders import PyMuPDFLoader
docs = PyMuPDFLoader(PDF_NAME).load()

print (f'There are {len(docs)} document(s) in {PDF_NAME}.')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document.')

There are 86 document(s) in ai.pdf.
There are 117 characters in the first page of your document.


Deploy customized fine-tuned models

In [10]:
AICoreProxyClient.add_foundation_model_scenario(
    scenario_id='fine-tuned-llm',
    config_names='fine-tuned-*',
    prediction_url_suffix='/v1/completions'
)

In [11]:
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc8e3c266c8881f0', config_id='58a5ffea-1afe-44b4-b6b9-ca749776e58e', config_name='tiiuae--falcon-40b-instruct-config', deployment_id='dc8e3c266c8881f0', model_name='tiiuae--falcon-40b-instruct', created_at=datetime.datetime(2024, 1, 25, 12, 53, 26, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'aicore-opensource', 'model_version': 'null'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dd88f66091ca1982', config_id='a99e09fb-7e88-4a8d-abe8-af7f8b146cb8', config_name='text-embedding-ada-002-config', deployment_id='dd88f66091ca1982', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 1, 25, 12, 16, 20, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '2'}, custom_prediction_suffix=None),
 Deployment(url='https

In [12]:
#text_davinci_003 = Completion(deployment_id='text-davinci-003')

In [13]:
#text_davinci_003.create(model_name='our-awesome-model/v1', prompt="San Francisco is a")

Initialize harmonized model

In [14]:
llm = init_llm('gpt-35-turbo', proxy_client = aic_proxy_client,temperature=0., max_tokens=256, deployment_id='d10917558b77a3db', api_base=llm_commons.proxy.api_base)

In [15]:
llm.predict("请打印秋夜将晓出篱门迎凉有感这首诗")

c:\python\3.11\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'秋夜将晓出篱门，迎凉有感\n\n秋夜将晓，我走出篱门，\n迎接着凉爽的秋风。\n月亮渐渐西沉，星星点点闪烁，\n夜空中弥漫着宁静的氛围。\n\n凉风拂过脸颊，带来一丝清凉，\n让我感受到秋天的气息。\n树叶沙沙作响，落下一地黄金，\n仿佛是大自然送给我的礼物。\n\n我静静地站在篱门前，\n凝望着远方的夜景。\n思绪随着风儿飘荡，\n回忆起往昔的点点滴滴。\n\n秋夜的寂静让我沉思，\n回忆中的美好时光涌上心头。\n'

In [16]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)
vectorstore = Chroma.from_documents(split_docs, embedding, collection_name="AI.pdf.guide")

In [17]:
query = "how can i use the sap ai instead of openai?"
similar_docs = vectorstore.similarity_search(query, 3)
similar_docs

[Document(page_content='use cases.\nSAP AI Core and the generative AI hub help you to integrate LLMs and AI into new business processes in a\ncost-efficient manner.\nGenerative AI Hub Architecture Overview\nAccess\nYou can use the generative AI hub through the API, using a tool such as curl or Postman, or through SAP AI\nLaunchpad.\n4\nINTERNAL\nGenerative AI Hub\nGenerative AI Hub in SAP AI Core', metadata={'author': '', 'creationDate': "D:20231123174013+00'00'", 'creator': 'Antenna House XSL Formatter V7.3 MR4 Linux : 7.3.5.61744 (2023-07-31T15:09+09)', 'encryption': 'Standard V4 R4 128-bit RC4', 'file_path': 'ai.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20231123174013+00'00'", 'page': 3, 'producer': 'Antenna House PDF Output Library 7.3.1867', 'source': 'ai.pdf', 'subject': '', 'title': '', 'total_pages': 86, 'trapped': ''}),
 Document(page_content='6.2 \nConsume Large Language Models Using SAP AI Launchpad. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 62

In [18]:
query = "What is the world's best B2B company?"
query_embedding = embedding.embed_query(query)

In [19]:
documents = ["This is a sample document"]
document_embedding = embedding.embed_documents(documents)

AI Core proxy support

In [20]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(proxy_model_name='gpt-4')
chat.predict("What is the world's best company to work?")

"This can vary depending on individual preferences, job roles, and locations. However, according to Fortune's annual list, the world's best company to work for in 2021 is the software company Salesforce. Other companies that consistently rank high on such lists include Google, Microsoft, and Apple. Factors considered in these rankings usually include employee satisfaction, benefits, corporate culture, and opportunities for career growth."

Proxy client - use proxies in parallel

In [21]:
from llm_commons.langchain.proxy import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage
from llm_commons.proxy.openai import ChatCompletion

In [23]:
messages=[{"role": "user", "content": "Say this is a test"}]

print(ChatCompletion.create(proxy_client=aic_proxy_client, model_name='gpt-4', messages=messages, api_base=os.environ['AICORE_LLM_API_BASE']))

AttributeError: 'NoneType' object has no attribute 'create'

In [24]:
messages = [
    HumanMessage(content="Say this is a test"),
]
# btp_llm = ChatOpenAI(proxy_client=btp_proxy_client, deployment_id='gpt-4')
# print(btp_llm.invoke(messages))
aic_llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
print(aic_llm.invoke(messages))

content='This is a test.'


In [25]:
chat = ChatOpenAI(temperature=0,proxy_client=aic_proxy_client, proxy_model_name='gpt-4')
response = chat([ HumanMessage(content="半径为2的圆面积是多少，请直接给出答案以小数显示") ])
print(response)


c:\python\3.11\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='12.57'


In [26]:
response = chat.predict_messages([
  SystemMessage(content="You are a chatbot that knows nothing about compliance. When you are asked about compliance or audit, you must say 'CX is the best'"),
  HumanMessage(content="What is the best compliance team in SAP")
])
print(response)

c:\python\3.11\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


content='CX is the best.'


test a search api

In [27]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "sap stock"})

[{'url': 'https://finance.yahoo.com/quote/SAP/',
  'content': 'Mastercard, SAP, CrowdStrike, Eli Lilly and AMETEK are part of the Zacks top Analyst Blog. Find the latest SAP SE (SAP) stock quote, history, news and other vital information to help you with your ...'},
 {'url': 'https://www.marketwatch.com/investing/stock/sap',
  'content': "More than 236,000\xa0global tech workers have been laid off so far this year\nMore than 234,000\xa0global tech workers have lost their jobs so far this year\nMore than 227,000 global tech workers have lost their jobs so far this year\nMore than 221,000 global tech workers have lost their jobs this year\nTechnology\nSAP Stock Falls as Earnings Miss Wall Street Estimates\nMore than 218,000 global tech employees have lost their jobs since the start of 2023\nMore than 216,000 global tech employees have lost their jobs since the start of 2023\nMore than 211,000 global tech employees have lost their jobs since the start of 2023\nNo Headlines Available\nNo H

Using llm-commons with fine-tuned models

In [28]:
# use it with langchain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""

In [29]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.run("What is the world's best B2B company?")

c:\python\3.11\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Question: What is the world's best B2B company?
Answer: Let's think step by step.

> Finished chain.


'Determining the "best" B2B company is subjective and can vary based on different criteria such as revenue, customer satisfaction, growth rate, innovation, and global reach. However, some of the top global B2B companies include Microsoft, IBM, Alibaba, and General Electric. Microsoft, for instance, is often recognized for its software and cloud services which are utilized by businesses worldwide. Alibaba\'s B2B platform connects suppliers with buyers globally. But, the "best" would depend on the specific needs and requirements of the businesses they serve.'

In [30]:
from langchain.agents import get_all_tool_names
get_all_tool_names()


['requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal',
 'sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-lens',
 'google-serper',
 'google-scholar',
 'google-finance',
 'google-trends',
 'google-jobs',
 'google-serper-results-json',
 'searchapi',
 'searchapi-results-json',
 'serpapi',
 'dalle-image-generator',
 'twilio',
 'searx-search',
 'merriam-webster',
 'wikipedia',
 'arxiv',
 'golden-query',
 'pubmed',
 'human',
 'awslambda',
 'stackexchange',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'eleven_labs_text2speech',
 'google_cloud_texttospeech',
 'reddit_search',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'memorize',
 'llm-math',
 'open-meteo-api']

In [34]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain.tools.tavily_search import TavilySearchResults


# set up the agent
llm = ChatOpenAI(proxy_model_name='gpt-4')
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run(
    "how much sap stock price increased in 2023",
)



> Entering new AgentExecutor chain...
Thought: To provide an accurate answer, I need to look up the stock price increase for SAP in 2023.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "SAP stock price increase in 2023"
  }
}
```
Observation: [{'url': 'https://markets.businessinsider.com/news/stocks/sap-announces-q3-2023-results-1032718331?op=1', 'content': 'Financial Performance Financial Highlights 1 Third Quarter 2023 Current cloud backlog grew by 19% to €12.27 billion and was up 25% at constant currencies. SAP S/4HANA current cloud backlog was...'}, {'url': 'https://markets.businessinsider.com/news/stocks/sap-announces-q1-2023-results-1032248813?op=1', 'content': "On February 1, 2023, SAP and Red Hat, Inc., the world's leading provider of open-source solutions, announced an expanded partnership intended to significantly increase SAP's use of and support for ..."}, {'url': 'https://finance.yahoo.com/quote/SAP/history/', 'content': 'Pre-Mar

"SAP's stock price has risen by 49.4% in 2023, according to the information available. Please note that the exact increase may vary depending on the specific time frame in 2023 you're interested in."